In [14]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [15]:
df = pd.read_csv('fake_or_real_news.csv')

In [16]:
# Pre-Processing
df['text'] = df['text'].apply(lambda x: x.lower())

In [17]:
import nltk
import re
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re,string,unicodedata

In [18]:
####
df['text'] = df['text'] + " " + df['title']
df=df.dropna()
df.reset_index(drop=True, inplace=True)

nltk.download('stopwords')
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

### Dataset Preprocessing
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
    
# Removing URL's
def remove_url(text):
    return re.sub(r'http\S+', '', text)

#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_url(text)
    text = remove_stopwords(text)
    return text

#Apply function on review column
df['text']=df['text'].apply(denoise_text)
####

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tanvi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'].values)
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :- ",vocab_size)
X = tokenizer.texts_to_sequences(df['text'].values)

Vocabulary Size :-  92127


In [20]:
max_length = 1000
# Padding
X = pad_sequences(X,maxlen = max_length, padding = 'post')

In [21]:
y = pd.get_dummies(df['label']).values

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33,random_state=53)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(4244, 1000) (4244, 2)
(2091, 1000) (2091, 2)


In [23]:
from numpy import array
from numpy import asarray
from numpy import zeros

# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.200d.txt',encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# Creating Embedding Matrix

# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 200))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


In [24]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 200, weights=[embedding_matrix], input_length=max_length, trainable=False))
model.add(Flatten())
model.add(Dense(2, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1000, 200)         18425400  
_________________________________________________________________
flatten_2 (Flatten)          (None, 200000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 400002    
Total params: 18,825,402
Trainable params: 400,002
Non-trainable params: 18,425,400
_________________________________________________________________
None


In [25]:
model.fit(X_train, y_train, epochs=50, verbose=0)

In [26]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 84.935439
